In [1]:
import json
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, LSTM, Dense,Embedding
import numpy as np

In [2]:
from keras.callbacks import EarlyStopping

In [3]:
with open('train.json', 'r', encoding='utf8') as f:
    data = json.load(f)

In [4]:
input_words = []
output_words = []
for key in data:
    input_words.append(key.lower())
    output_words.append(data[key][0].lower())

In [5]:
input_chars = sorted(list(set(''.join(input_words))))
input_char_indices = dict((c, i) for i, c in enumerate(input_chars))
input_indices_char = dict((i, c) for i, c in enumerate(input_chars))

output_chars = sorted(list(set(''.join(output_words))))
output_char_indices = dict((c, i) for i, c in enumerate(output_chars))
output_indices_char = dict((i, c) for i, c in enumerate(output_chars))


In [6]:
len(input_chars)

43

In [7]:
len(input_words)

12937

In [9]:
with open('input_char_indices.json1', 'w') as f:
    json.dump(input_char_indices, f)

with open('output_char_indices.json1', 'w') as f:
    json.dump(output_char_indices, f)

with open('input_indices_char.json1', 'w') as f:
    json.dump(input_indices_char, f)

with open('output_indices_char.json1', 'w') as f:
    json.dump(output_indices_char, f)

In [10]:
max_encoder_seq_length = max([len(word) for word in input_words])
max_decoder_seq_length = max([len(word) for word in output_words])

In [11]:
max_decoder_seq_length

62

In [12]:
max_encoder_seq_length = max([len(word) for word in input_words])
max_decoder_seq_length = max([len(word) for word in output_words])

encoder_input_data = np.zeros((len(input_words), max_encoder_seq_length, len(input_chars)), dtype='float32')
decoder_input_data = np.zeros((len(input_words), max_decoder_seq_length, len(output_chars)), dtype='float32')
decoder_target_data = np.zeros((len(input_words), max_decoder_seq_length, len(output_chars)), dtype='float32')

for i, (input_word, output_word) in enumerate(zip(input_words, output_words)):
    for t, char in enumerate(input_word):
        encoder_input_data[i, t, input_char_indices[char]] = 1.
    for t, char in enumerate(output_word):
        decoder_input_data[i, t, output_char_indices[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, output_char_indices[char]] = 1.


In [13]:
batch_size = 64
epochs = 50
latent_dim = 256

encoder_inputs = Input(shape=(None, len(input_chars)))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, len(output_chars)))
decoder = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(len(output_chars), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')


In [14]:
for epoch in range(epochs):
    print("Epoch:", epoch + 1)
    for i in range(0, len(encoder_input_data), batch_size):
        encoder_input_batch = encoder_input_data[i:i + batch_size]
        decoder_input_batch = decoder_input_data[i:i + batch_size]
        decoder_target_batch = decoder_target_data[i:i + batch_size]
        loss = model.train_on_batch(
            [encoder_input_batch, decoder_input_batch],
            decoder_target_batch)
        print("Batch:", i + 1, "Loss:", loss)

Epoch: 1
Batch: 1 Loss: 0.4850185811519623
Batch: 65 Loss: 0.5584812760353088
Batch: 129 Loss: 0.4856483042240143
Batch: 193 Loss: 0.7069913148880005
Batch: 257 Loss: 0.470507949590683
Batch: 321 Loss: 0.457257479429245
Batch: 385 Loss: 0.4632970094680786
Batch: 449 Loss: 0.40953290462493896
Batch: 513 Loss: 0.431883841753006
Batch: 577 Loss: 0.4343436658382416
Batch: 641 Loss: 0.5240647792816162
Batch: 705 Loss: 0.44022560119628906
Batch: 769 Loss: 0.5230588316917419
Batch: 833 Loss: 0.36939725279808044
Batch: 897 Loss: 0.6504051685333252
Batch: 961 Loss: 0.3948502242565155
Batch: 1025 Loss: 0.4375607371330261
Batch: 1089 Loss: 0.421008437871933
Batch: 1153 Loss: 0.4713257849216461
Batch: 1217 Loss: 0.43104884028434753
Batch: 1281 Loss: 0.46846911311149597
Batch: 1345 Loss: 0.4768661558628082
Batch: 1409 Loss: 0.40034762024879456
Batch: 1473 Loss: 0.44411081075668335
Batch: 1537 Loss: 0.41657957434654236
Batch: 1601 Loss: 0.35593700408935547
Batch: 1665 Loss: 0.3959711194038391
Batch:

Batch: 1089 Loss: 0.4177849590778351
Batch: 1153 Loss: 0.46188297867774963
Batch: 1217 Loss: 0.41874054074287415
Batch: 1281 Loss: 0.4535689949989319
Batch: 1345 Loss: 0.46112361550331116
Batch: 1409 Loss: 0.3922782242298126
Batch: 1473 Loss: 0.4388578236103058
Batch: 1537 Loss: 0.41097012162208557
Batch: 1601 Loss: 0.3536154329776764
Batch: 1665 Loss: 0.38978806138038635
Batch: 1729 Loss: 0.4714527726173401
Batch: 1793 Loss: 0.358114629983902
Batch: 1857 Loss: 0.4252123236656189
Batch: 1921 Loss: 0.49025166034698486
Batch: 1985 Loss: 0.3858853578567505
Batch: 2049 Loss: 0.3662634789943695
Batch: 2113 Loss: 0.4173298478126526
Batch: 2177 Loss: 0.4668968617916107
Batch: 2241 Loss: 0.5193275809288025
Batch: 2305 Loss: 0.5513496398925781
Batch: 2369 Loss: 0.5526253581047058
Batch: 2433 Loss: 0.4704301655292511
Batch: 2497 Loss: 0.5314379930496216
Batch: 2561 Loss: 0.45921337604522705
Batch: 2625 Loss: 0.4598776400089264
Batch: 2689 Loss: 0.4724312126636505
Batch: 2753 Loss: 0.399548977613

Batch: 2113 Loss: 0.40957486629486084
Batch: 2177 Loss: 0.45868560671806335
Batch: 2241 Loss: 0.509031355381012
Batch: 2305 Loss: 0.5383694171905518
Batch: 2369 Loss: 0.5405889749526978
Batch: 2433 Loss: 0.4625864326953888
Batch: 2497 Loss: 0.5216218829154968
Batch: 2561 Loss: 0.45349010825157166
Batch: 2625 Loss: 0.4567578136920929
Batch: 2689 Loss: 0.46924853324890137
Batch: 2753 Loss: 0.39526882767677307
Batch: 2817 Loss: 0.5839964747428894
Batch: 2881 Loss: 0.5101712942123413
Batch: 2945 Loss: 0.4458980858325958
Batch: 3009 Loss: 0.6122115254402161
Batch: 3073 Loss: 0.48675867915153503
Batch: 3137 Loss: 0.4414893090724945
Batch: 3201 Loss: 0.41236433386802673
Batch: 3265 Loss: 0.39437466859817505
Batch: 3329 Loss: 0.42857104539871216
Batch: 3393 Loss: 0.4736706018447876
Batch: 3457 Loss: 0.4375152587890625
Batch: 3521 Loss: 0.3275603652000427
Batch: 3585 Loss: 0.3176950216293335
Batch: 3649 Loss: 0.5352853536605835
Batch: 3713 Loss: 0.43686428666114807
Batch: 3777 Loss: 0.464833647

Batch: 3201 Loss: 0.4090007543563843
Batch: 3265 Loss: 0.3913416564464569
Batch: 3329 Loss: 0.42457109689712524
Batch: 3393 Loss: 0.4679761826992035
Batch: 3457 Loss: 0.4317532777786255
Batch: 3521 Loss: 0.325680673122406
Batch: 3585 Loss: 0.31638845801353455
Batch: 3649 Loss: 0.5267871618270874
Batch: 3713 Loss: 0.43194013833999634
Batch: 3777 Loss: 0.45940670371055603
Batch: 3841 Loss: 0.45819246768951416
Batch: 3905 Loss: 0.40485820174217224
Batch: 3969 Loss: 0.5686449408531189
Batch: 4033 Loss: 0.4279710650444031
Batch: 4097 Loss: 0.41871145367622375
Batch: 4161 Loss: 0.4912349581718445
Batch: 4225 Loss: 0.3569352328777313
Batch: 4289 Loss: 0.38253888487815857
Batch: 4353 Loss: 0.5247988700866699
Batch: 4417 Loss: 0.7033610939979553
Batch: 4481 Loss: 0.7377933263778687
Batch: 4545 Loss: 0.5922683477401733
Batch: 4609 Loss: 0.39232486486434937
Batch: 4673 Loss: 0.36803752183914185
Batch: 4737 Loss: 0.48434436321258545
Batch: 4801 Loss: 0.3976460099220276
Batch: 4865 Loss: 0.40695744

Batch: 4289 Loss: 0.3791562616825104
Batch: 4353 Loss: 0.5161954760551453
Batch: 4417 Loss: 0.6907860636711121
Batch: 4481 Loss: 0.7347877025604248
Batch: 4545 Loss: 0.585457444190979
Batch: 4609 Loss: 0.38826659321784973
Batch: 4673 Loss: 0.367742657661438
Batch: 4737 Loss: 0.4802986979484558
Batch: 4801 Loss: 0.394662082195282
Batch: 4865 Loss: 0.4037718176841736
Batch: 4929 Loss: 0.4530487656593323
Batch: 4993 Loss: 0.5746557116508484
Batch: 5057 Loss: 0.5514033436775208
Batch: 5121 Loss: 0.4287444055080414
Batch: 5185 Loss: 0.918160080909729
Batch: 5249 Loss: 0.39441612362861633
Batch: 5313 Loss: 0.4709579050540924
Batch: 5377 Loss: 0.44668492674827576
Batch: 5441 Loss: 0.5564667582511902
Batch: 5505 Loss: 0.5123625993728638
Batch: 5569 Loss: 0.5475814342498779
Batch: 5633 Loss: 0.3989991843700409
Batch: 5697 Loss: 0.4668348431587219
Batch: 5761 Loss: 0.587019681930542
Batch: 5825 Loss: 0.4551815688610077
Batch: 5889 Loss: 0.3984215557575226
Batch: 5953 Loss: 0.4027496576309204
Bat

...

Batch: 5569 Loss: 0.5211613774299622
Batch: 5633 Loss: 0.3770032227039337
Batch: 5697 Loss: 0.427537202835083
Batch: 5761 Loss: 0.5466140508651733
Batch: 5825 Loss: 0.425060510635376
Batch: 5889 Loss: 0.3801485598087311
Batch: 5953 Loss: 0.3827598989009857
Batch: 6017 Loss: 0.3181201219558716
Batch: 6081 Loss: 0.2942960858345032
Batch: 6145 Loss: 0.4018620252609253
Batch: 6209 Loss: 0.34119346737861633
Batch: 6273 Loss: 0.45523375272750854
Batch: 6337 Loss: 0.33007580041885376
Batch: 6401 Loss: 0.33376479148864746
Batch: 6465 Loss: 0.3730441629886627
Batch: 6529 Loss: 0.3519212007522583
Batch: 6593 Loss: 0.37082812190055847
Batch: 6657 Loss: 0.30801600217819214
Batch: 6721 Loss: 0.41355976462364197
Batch: 6785 Loss: 0.4082566499710083
Batch: 6849 Loss: 0.413469135761261
Batch: 6913 Loss: 0.3903532326221466
Batch: 6977 Loss: 0.4247344136238098
Batch: 7041 Loss: 0.4139726161956787
Batch: 7105 Loss: 0.3468983471393585
Batch: 7169 Loss: 0.3928382694721222
Batch: 7233 Loss: 0.35076788067817

Batch: 6593 Loss: 0.370140016078949
Batch: 6657 Loss: 0.3091422915458679
Batch: 6721 Loss: 0.4211236834526062
Batch: 6785 Loss: 0.4073486030101776
Batch: 6849 Loss: 0.4093124270439148
Batch: 6913 Loss: 0.3905157744884491
Batch: 6977 Loss: 0.42445147037506104
Batch: 7041 Loss: 0.41518333554267883
Batch: 7105 Loss: 0.3284432291984558
Batch: 7169 Loss: 0.3923080265522003
Batch: 7233 Loss: 0.3507133722305298
Batch: 7297 Loss: 0.41097787022590637
Batch: 7361 Loss: 0.38229236006736755
Batch: 7425 Loss: 0.4651913046836853
Batch: 7489 Loss: 0.5585435628890991
Batch: 7553 Loss: 0.36369845271110535
Batch: 7617 Loss: 0.3634454607963562
Batch: 7681 Loss: 0.43997281789779663
Batch: 7745 Loss: 0.45146504044532776
Batch: 7809 Loss: 0.41565439105033875
Batch: 7873 Loss: 0.33000773191452026
Batch: 7937 Loss: 0.538555383682251
Batch: 8001 Loss: 0.38976603746414185
Batch: 8065 Loss: 0.5330522656440735
Batch: 8129 Loss: 0.3918055295944214
Batch: 8193 Loss: 0.43914350867271423
Batch: 8257 Loss: 0.466764628

Batch: 7681 Loss: 0.452227920293808
Batch: 7745 Loss: 0.45846042037010193
Batch: 7809 Loss: 0.4206087589263916
Batch: 7873 Loss: 0.3293689489364624
Batch: 7937 Loss: 0.535714864730835
Batch: 8001 Loss: 0.3905866742134094
Batch: 8065 Loss: 0.5358048677444458
Batch: 8129 Loss: 0.3928598165512085
Batch: 8193 Loss: 0.45920881628990173
Batch: 8257 Loss: 0.4637144207954407
Batch: 8321 Loss: 0.6343038082122803
Batch: 8385 Loss: 0.31888115406036377
Batch: 8449 Loss: 0.3547744154930115
Batch: 8513 Loss: 0.6352112293243408
Batch: 8577 Loss: 0.3247746527194977
Batch: 8641 Loss: 0.4904254972934723
Batch: 8705 Loss: 0.3898472189903259
Batch: 8769 Loss: 0.4609697163105011
Batch: 8833 Loss: 0.5118924379348755
Batch: 8897 Loss: 0.7818402647972107
Batch: 8961 Loss: 0.3954712152481079
Batch: 9025 Loss: 0.39483073353767395
Batch: 9089 Loss: 0.40142810344696045
Batch: 9153 Loss: 0.4609338641166687
Batch: 9217 Loss: 0.48834171891212463
Batch: 9281 Loss: 0.4436900317668915
Batch: 9345 Loss: 0.34964290261268

Batch: 8769 Loss: 0.45675402879714966
Batch: 8833 Loss: 0.5101260542869568
Batch: 8897 Loss: 0.7808215022087097
Batch: 8961 Loss: 0.39105063676834106
Batch: 9025 Loss: 0.38841408491134644
Batch: 9089 Loss: 0.39781585335731506
Batch: 9153 Loss: 0.45750245451927185
Batch: 9217 Loss: 0.48772868514060974
Batch: 9281 Loss: 0.4437682330608368
Batch: 9345 Loss: 0.3499528169631958
Batch: 9409 Loss: 0.4822733700275421
Batch: 9473 Loss: 0.5407877564430237
Batch: 9537 Loss: 0.463338166475296
Batch: 9601 Loss: 0.3544165790081024
Batch: 9665 Loss: 0.3337952494621277
Batch: 9729 Loss: 0.43235310912132263
Batch: 9793 Loss: 0.3762480616569519
Batch: 9857 Loss: 0.4302811920642853
Batch: 9921 Loss: 0.39237818121910095
Batch: 9985 Loss: 0.42949187755584717
Batch: 10049 Loss: 0.4964306950569153
Batch: 10113 Loss: 0.38697361946105957
Batch: 10177 Loss: 0.4680977165699005
Batch: 10241 Loss: 0.36260655522346497
Batch: 10305 Loss: 0.4362812340259552
Batch: 10369 Loss: 0.4799765944480896
Batch: 10433 Loss: 0.4

Batch: 9857 Loss: 0.428533136844635
Batch: 9921 Loss: 0.39178135991096497
Batch: 9985 Loss: 0.4290997385978699
Batch: 10049 Loss: 0.4958685636520386
Batch: 10113 Loss: 0.38581472635269165
Batch: 10177 Loss: 0.46941715478897095
Batch: 10241 Loss: 0.3630105257034302
Batch: 10305 Loss: 0.4364536702632904
Batch: 10369 Loss: 0.4799461364746094
Batch: 10433 Loss: 0.3994373083114624
Batch: 10497 Loss: 0.39129191637039185
Batch: 10561 Loss: 0.3672262728214264
Batch: 10625 Loss: 0.4307982921600342
Batch: 10689 Loss: 0.4002099633216858
Batch: 10753 Loss: 0.3564082384109497
Batch: 10817 Loss: 0.31119754910469055
Batch: 10881 Loss: 0.37140870094299316
Batch: 10945 Loss: 0.3796878457069397
Batch: 11009 Loss: 0.40737292170524597
Batch: 11073 Loss: 0.4359918534755707
Batch: 11137 Loss: 0.3480255901813507
Batch: 11201 Loss: 0.3631518483161926
Batch: 11265 Loss: 0.5100896954536438
Batch: 11329 Loss: 0.63584965467453
Batch: 11393 Loss: 0.2811521291732788
Batch: 11457 Loss: 0.3569473922252655
Batch: 1152

In [15]:
model.save('transliteration_modelFinal2.h5')

In [23]:
import numpy as np
import json
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# load the character dictionaries
with open('input_char_indices.json', 'r') as f:
    input_char_indices = json.load(f)

with open('output_char_indices.json', 'r') as f:
    output_char_indices = json.load(f)

with open('input_indices_char.json', 'r') as f:
    input_indices_char = json.load(f)

with open('output_indices_char.json', 'r') as f:
    output_indices_char = json.load(f)

# define the maximum lengths of input and output sequences
max_encoder_seq_length = 61
max_decoder_seq_length = 62

# load the trained model
model = load_model('transliteration_modelFinal2.h5')

# define the encoder model
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

# define the decoder model
decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = Input(shape=(256,), name='input_3')
decoder_state_input_c = Input(shape=(256,), name='input_4')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

# define a function to perform the actual translation
def predict_transliteration(input_word):
    # convert the input word to the corresponding sequence of integers
    encoder_input_data = np.zeros((1, max_encoder_seq_length, len(input_char_indices)), dtype='float32')
    for t, char in enumerate(input_word):
        encoder_input_data[0, t, input_char_indices[char]] = 1.
    # encode the input sequence
    states_value = encoder_model.predict(encoder_input_data)
    # create an empty target sequence of length 1
    target_seq = np.zeros((1, 1, len(output_char_indices)))
    # set the first character of the target sequence as the start-of-sequence character
    target_seq[0, 0, output_char_indices[',']] = 1.
    # predict the next character until the end-of-sequence character is generated or the maximum length is reached
    output_word = ''
    for t in range(1, max_decoder_seq_length):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        # select the most probable character
        sampled_output_char = input_indices_char[str(np.argmax(output_tokens[0, -1, :]))]
        if sampled_output_char == '\n':
            break
        output_word += sampled_output_char
        # update the target sequence
        target_seq = np.zeros((1, 1, len(output_char_indices)))
        target_seq[0, 0, np.argmax(output_tokens[0, -1, :])] = 1.
        # update states
        states_value = [h, c]
    return output_word